In [8]:
import tensorflow as tf
import pickle
import os
import shutil
import tempfile
import datetime
from glob import iglob
import time
# import attention
from collections import deque
import pickle
from BeamSearchTree import BeamSearchTreeNode
import pyreader
import numpy as np
from attentioncell import attention_masks
from attention import Batcher, construct_feed_dict, extract_results, get_evals,AttentionModel,get_initial_state

In [9]:
def find_path(tree, k=1):

    paths = []

    def search_tree(path, node, path_prob):
        if not node.children:
            paths.append((path, path_prob))
        else:
            for child in node.children:
                search_tree(path + [child.token_id], child, path_prob + np.log(child.probability))

    search_tree([], tree, 0)

    sorted_paths = sorted(paths, key=lambda x: x[1], reverse=True)
    return [path[0] for path in sorted_paths[:k]]

In [10]:
type_max_rands = {
    "var": 4750, "function": 2900, "Class": 440, "attribute": 2400, "arg": 2400
}
def map_token(word_to_id, token):
    mask = 0
    if token.startswith("(*) "):
        mask = 1
        token = token.replace("(*) ", "")

    if token in word_to_id:
        return word_to_id[token], mask

    # Not in map, is it an identifier?
    if "|" in token:
        
        spl = token.split("|")
#         print(token, spl)
        if spl[1] in word_to_id:
            return word_to_id[spl[1]]
        elif spl[0] in word_to_id:
            return word_to_id[spl[0]]
#         return pyreader.oov_id

    elif any([token.startswith(prefix) for prefix in [key for key in type_max_rands]]):
        return pyreader.oov_id

    raise KeyError(token)

In [11]:
seq_length = 1
batch_size = 1
hidden_size = 50
num_samples = 3
attention_num = 5
lambda_type='state'
max_attention=10
model_path = './model/2017-07-04--23-13--629882'
data_path='data_samples/mapping.map'
with open(data_path, "rb") as f:
    word_to_id = pickle.load(f)
vocab_size = len(word_to_id)
inv_map = {v: k for k, v in word_to_id.items()}

In [12]:

map_token(word_to_id, 'attribute|attribute1729')

13883

In [13]:
# all_test_cases = ["import", "os", "\n", "class", "(*) Class291", ":", "\n", "§<indent>§", "def", "__init__", "(", "self", ",",
#      "(*) arg123", ")", ":", "\n", "§<indent>§", "self", ".", "(*) attribute|attribute172", "=", "arg123", "\n", "\n",
#      "§<dedent>§",
#      "def", "(*) function234", "(", "self", ",", "(*) filename|arg432", ")", ":", "\n", "§<indent>§", "with", "open",
#      "(", "filename|arg432", ",", "'r'", ")", "as", "(*) f|var76", ":", "\n", "§<indent>§", "(*) lines|var91", "=",
#      "f|var76", ".",
#      "readlines", "(", ")", "\n", "§<dedent>§", "return", "len", "(", "lines|var91", ")", "\n", "\n", "§<dedent>§",
#      "def", "(*) func|function921", "(", "self", ",", "(*) arg191", ")", ":", "\n", "§<indent>§", "(*) var543", "=",
#      "os", ".", "path", ".", "join", "(", "self", "."]
all_test_cases = [
    ["import", "numpy", "as", "np", "\n", "(*) data", "=", "np", ".", "array", "(", "[", "§NUM§", ",", "§NUM§", ",", "§NUM§", ",", "§NUM§", ",", "]", ")", "\n", "np", "."],
    ["import", "os", "\n", "\n", "class", "Class253", ":", "\n", "\n", "§<indent>§",
     "def", "__init__", "(", "self", ",", "arg651", ")", ":", "\n", "§<indent>§",
     "self", ".", "attribute943", "=", "arg651", "\n", "\n", "§<dedent>§",
     "def", "function1690", "(", "self", ",", "arg2004", ")", ":", "\n", "§<indent>§",
     "var4040", "=", "os", ".", "path", ".", "join", "(", "self", ".", "attribute943", ",", "arg2004", ")", "\n",
     "print", "(", "§OOV§", "%", "(", "str", "("
     ],
    ["import", "os", "\n", "class", "(*) MyClass|Class291", ":", "\n", "§<indent>§", "def", "__init__", "(", "self", ",", "(*) arg123", ")", ":", "\n", "§<indent>§", "self", ".", "(*) attribute|attribute1729", "=", "arg123", "\n", "\n", "§<dedent>§",
    "def", "(*) function234", "(", "self", ",", "(*) filename|arg432", ")", ":", "\n", "§<indent>§", "with", "open", "(", "filename|arg432", ",", "'r'", ")", "as", "(*) f|var76", ":", "\n", "§<indent>§", "(*) lines|var91", "=", "f|var76", ".",
    "readlines", "(", ")", "\n", "§<dedent>§", "return", "len", "(", "lines|var91", ")", "\n", "\n", "§<dedent>§", "def", "(*) func|function921", "(", "self", ",", "(*) arg191", ")", ":", "\n", "§<indent>§", "(*) var543", "=", "os", ".", "path", ".", "join", "(", "self", "."],
    ["class", "(*) Class210", ":", "\n", "\n", "§<indent>§", "def", "__init__", "(", "self", ",",
     "(*) arg233", ")", ":", "\n",
     "§<indent>§",
     "self", ".", "(*) attribute1729", "=", "arg233", "\n",
     "\n", "§<dedent>§", "def", "(*) function1234", "(", "self", ",", "(*) arg635", ")", ":", "\n", "§<indent>§",
     "return", "§OOV§", "if", "arg635", "else", "§OOV§", "\n",
     "\n", "§<dedent>§", "def", "(*) function651", "(", "self", ",", "(*) arg536", ")", ":", "\n", "§<indent>§",
     "return", "§OOV§", "if", "arg536", "else", "§OOV§", "\n",
     "\n", "§<dedent>§", "def", "(*) function2766", "(", "self", ",", "(*) arg1556", ")", ":", "\n",
     "§<indent>§", "(*) var155", "=", "os", ".", "path", ".", "join", "(", "arg1556", ",", "self", "."],
    ["def", "(*) function20", "(", "(*) arg123", ")", ":", "\n", "§<indent>§", "print", "("],
    ["class", "(*) Class12", ":", "\n", "§<indent>§", "def", "__init__", "(", "self", ")", ":", "\n", "§<indent>§",
     "self", ".", "(*) attribute462", "=", "§OOV§", "\n", "\n", "§<dedent>§", "§<dedent>§", "if", "__name__", "==",
     "'__main__'", ":", "\n", "§<indent>§", "(*) var821", "="],
    ["def", "(*) function20", "(", "(*) arg123", ")", ":", "\n", "§<indent>§", "pass", "\n", "\n", "(*) var821", "="],
    ["import", "os", "\n", "\n", "def", "(*) function23", "(", "(*) arg123", ")", ":", "\n", "§<indent>§", "os", ".",
     "path", "."],
    ["import", "os", "\n", "\n", "def", "(*) function23", "(", "(*) arg123", ")", ":", "\n", "§<indent>§", "os", ".",
     "path", ".", "join", "("],
    ["(*) var464", "=", "[", "§NUM§", "§NUM§", "§NUM§", "]", "\n", "(*) var921", "=", "sorted", "("],
    ["import", "os", "\n", "class", "(*) Class291", ":", "\n", "§<indent>§", "def", "__init__", "(", "self", ",",
     "(*) arg123", ")", ":", "\n", "§<indent>§", "self", ".", "(*) attribute|attribute1729", "=", "arg123", "\n", "\n",
     "§<dedent>§",
     "def", "(*) function234", "(", "self", ",", "(*) filename|arg432", ")", ":", "\n", "§<indent>§", "with", "open",
     "(", "filename|arg432", ",", "'r'", ")", "as", "(*) f|var76", ":", "\n", "§<indent>§", "(*) lines|var91", "=",
     "f|var76", ".",
     "readlines", "(", ")", "\n", "§<dedent>§", "return", "len", "(", "lines|var91", ")", "\n", "\n", "§<dedent>§",
     "def", "(*) func|function921", "(", "self", ",", "(*) arg191", ")", ":", "\n", "§<indent>§", "(*) var543", "=",
     "os", ".", "path", ".", "join", "(", "self", "."],
                 ['import', 'django'],
    ['import','datetime','\n', 'import','json','\n','import','os','\n','\n']
    ]

In [14]:
with tf.Graph().as_default(), tf.Session() as session:
#     masks_ = tf.placeholder(tf.bool, [seq_length, batch_size, attention_num], name="masks")
    masks_ = tf.placeholder(tf.bool, [seq_length, batch_size, attention_num], name="masks")
    input_data_ = tf.placeholder(tf.int32, [seq_length, batch_size], name="inputs")
    targets_ = tf.placeholder(tf.float32, [seq_length, batch_size], name="targets")
    
    a = AttentionModel(input_data=input_data_,
                                 targets=targets_,
                                 masks=masks_,
                                 is_training=False,
                                 attention_num= attention_num,
                                 batch_size=batch_size,
                                 hidden_size=hidden_size,
                                 num_samples=num_samples,
                                 seq_length=seq_length,
                                 vocab_size=vocab_size,
                                 lambda_type=lambda_type,
                                 max_attention=max_attention)
    
    
    variables = tf.trainable_variables()
    session.run(tf.global_variables_initializer())
    saver = tf.train.Saver(variables)
    saver.restore(session, os.path.join(model_path, "model.tf"))
    
    prediction_op = tf.nn.top_k(a.predict, 5)
    
    to_eval = [prediction_op[0], prediction_op[1]]
    evaluation = get_evals(to_eval, a)
    
    
    def run_network(token_id, state, att_states, att_ids, att_counts):
#         att_mask = np.ones([1, 1])
#         att_mask = attention_masks(1,np.array([0]),5)
        att_mask = attention_masks(1,np.zeros([5]),5)

        if isinstance(token_id, tuple):
            data = (np.array([[token_id[0]]]), np.array([[1]]), np.array([att_mask]), np.array([1]), np.array([1]))
        else:
            data = (np.array([[token_id]]), np.array([[1]]), np.array([att_mask]), np.array([1]), np.array([1]))
        feed_dict, _ = construct_feed_dict(a, data, state, att_states, att_ids, att_counts)

        results = session.run(evaluation, feed_dict)
        results, state, att_states, att_ids, alpha_states, att_counts, _ = extract_results(results, evaluation, 2,
                                                                                           a)
        return results, state, att_states, att_counts

    def beam(tree_node):
        # Populate the children of tree_node
        init_state, init_att_states, init_att_counts = tree_node.state
        results, state, att_states, att_counts = run_network(tree_node.token_id, init_state, init_att_states,
                                                             att_ids, init_att_counts)
        probs = results[0]
        predict_ids = results[1]
        for i in range(5):
            tree_node.add_child(BeamSearchTreeNode(predict_ids[0, i],attention_masks(1, np.zeros([5]), 5),
                                                   (state, att_states, att_counts), probs[0, i]))

    def beam_search_recursive(tree, current_depth):
        if current_depth < 2:
            for child in tree.children:
                beam(child)
                beam_search_recursive(child, current_depth + 1)
    
    
    
    state, att_states, att_ids, att_counts = get_initial_state(a, session)
    count=0
    accuracy=0
    for j, test_case in enumerate(all_test_cases):
        for i, token in enumerate(test_case[:-2]):
#             lst =np.ones([1, 1])
#             lst = attention_masks(1,np.array([0]),5)
            lst = attention_masks(1,np.zeros([5]),5)
            token_id = map_token(word_to_id, token)

            if isinstance(token_id, tuple):
                data = (np.array([[token_id[0]]]), np.array([[1]]), np.array([lst]), np.array([1]), np.array([1]))
            else:
                data = (np.array([[token_id]]), np.array([[1]]), np.array([lst]), np.array([1]), np.array([1]))
            results, state, att_states, att_counts = run_network(map_token(word_to_id, token), state, att_states,
                                                                         att_ids, att_counts)
            root = BeamSearchTreeNode(map_token(word_to_id, test_case[-1]),attention_masks(1,np.zeros([5]),5),
                                              (state, att_states, att_counts), 1)
            beam(root)
            beam_search_recursive(root, 1)
            path = find_path(root)[0]  # The most likely path
            actual = [map_token(word_to_id, t) for t in test_case[i + 1:i + 5 + 1]]

            pred = " ".join([inv_map[t].replace("\n", "<newline>") for t in path])
            pred = []
            for t in path:
                pred.append(inv_map[t].replace("\n", "<newline>"))
            pred_string = ' '.join(pred)

            act = []
            for t in actual:
                try:
                    act.append(inv_map[t[0]].replace("\n", "<newline>"))
                except:
#                     act.append(inv_map[t].replace("\n", "<newline>"))
                    continue
            act_str = ' '.join(act)
        
            print('-'*40)
            print("Token(input): ", token.replace("\n", "<newline>"))
            print("Predicted: ", pred_string)
            print('Actual   : ', act_str)
            
            for k in zip(pred, act):
                if k[0] == k[1] :
                    accuracy = accuracy+1
            count=count+1
    print('정답갯수 ' ,accuracy ,' 총 갯수 ',count)
    print("Accuracy: %f" % (accuracy / count))
#         results = session.run(evaluation, feed_dict)
#         results_match = ''.join(str(results[1][0][0]))
#         actual = [map_token(word_to_id, t) for t in all_test_cases[i + 1:i + 5 + 1]]
#         act = " ".join([inv_map[t[0]].replace("\n", "<newline>") for t in actual])
#         print('=============')
#         print('predict', inv_map[results[1][0][1]])
#         print('real', act)

Constructing Attention Cell
INFO:tensorflow:Restoring parameters from ./model/2017-07-04--23-13--629882\model.tf
----------------------------------------
Token(input):  import
Predicted:  §OOV§ (
Actual   :  numpy as np <newline> data
----------------------------------------
Token(input):  numpy
Predicted:  §OOV§ (
Actual   :  as np <newline> data =
----------------------------------------
Token(input):  as
Predicted:  §OOV§ (
Actual   :  np <newline> data = np
----------------------------------------
Token(input):  np
Predicted:  random .
Actual   :  <newline> data = np .
----------------------------------------
Token(input):  <newline>
Predicted:  random .
Actual   :  data = np . array
----------------------------------------
Token(input):  (*) data
Predicted:  create (
Actual   :  = np . array (
----------------------------------------
Token(input):  =
Predicted:  §OOV§ (
Actual   :  np . array ( [
----------------------------------------
Token(input):  np
Predicted:  §OOV§ (
Actual

----------------------------------------
Token(input):  arg2004
Predicted:  ) <newline>
Actual   :  ) <newline> print ( §OOV§
----------------------------------------
Token(input):  )
Predicted:  ) <newline>
Actual   :  <newline> print ( §OOV§ %
----------------------------------------
Token(input):  <newline>
Predicted:  os .
Actual   :  print ( §OOV§ % (
----------------------------------------
Token(input):  print
Predicted:  §OOV§ ,
Actual   :  ( §OOV§ % ( str
----------------------------------------
Token(input):  (
Predicted:  §OOV§ ,
Actual   :  §OOV§ % ( str (
----------------------------------------
Token(input):  §OOV§
Predicted:  ) )
Actual   :  % ( str (
----------------------------------------
Token(input):  %
Predicted:  §OOV§ ,
Actual   :  ( str (
----------------------------------------
Token(input):  (
Predicted:  §OOV§ ,
Actual   :  str (
----------------------------------------
Token(input):  import
Predicted:  through )
Actual   :  os <newline> class :
-------------

----------------------------------------
Token(input):  )
Predicted:  create (
Actual   :  <newline> <newline> §<dedent>§ def
----------------------------------------
Token(input):  <newline>
Predicted:  §OOV§ (
Actual   :  <newline> §<dedent>§ def (
----------------------------------------
Token(input):  <newline>
Predicted:  assertEqual (
Actual   :  §<dedent>§ def ( self
----------------------------------------
Token(input):  §<dedent>§
Predicted:  assertEqual (
Actual   :  def ( self ,
----------------------------------------
Token(input):  def
Predicted:  save (
Actual   :  ( self , arg191
----------------------------------------
Token(input):  (*) func|function921
Predicted:  create (
Actual   :  ( self , arg191 )
----------------------------------------
Token(input):  (
Predicted:  §OOV§ )
Actual   :  self , arg191 ) :
----------------------------------------
Token(input):  self
Predicted:  §OOV§ )
Actual   :  , arg191 ) : <newline>
----------------------------------------
Token

----------------------------------------
Token(input):  )
Predicted:  create (
Actual   :  : <newline> §<indent>§ return §OOV§
----------------------------------------
Token(input):  :
Predicted:  §OOV§ :
Actual   :  <newline> §<indent>§ return §OOV§ if
----------------------------------------
Token(input):  <newline>
Predicted:  §OOV§ (
Actual   :  §<indent>§ return §OOV§ if arg536
----------------------------------------
Token(input):  §<indent>§
Predicted:  §OOV§ (
Actual   :  return §OOV§ if arg536 else
----------------------------------------
Token(input):  return
Predicted:  §OOV§ (
Actual   :  §OOV§ if arg536 else §OOV§
----------------------------------------
Token(input):  §OOV§
Predicted:  format (
Actual   :  if arg536 else §OOV§ <newline>
----------------------------------------
Token(input):  if
Predicted:  §OOV§ (
Actual   :  arg536 else §OOV§ <newline> <newline>
----------------------------------------
Token(input):  arg536
Predicted:  §OOV§ (
Actual   :  else §OOV§ <new

----------------------------------------
Token(input):  (*) function20
Predicted:  §OOV§ (
Actual   :  ( arg123 ) : <newline>
----------------------------------------
Token(input):  (
Predicted:  §OOV§ )
Actual   :  arg123 ) : <newline> §<indent>§
----------------------------------------
Token(input):  (*) arg123
Predicted:  §NUM§ )
Actual   :  ) : <newline> §<indent>§ pass
----------------------------------------
Token(input):  )
Predicted:  self .
Actual   :  : <newline> §<indent>§ pass <newline>
----------------------------------------
Token(input):  :
Predicted:  self .
Actual   :  <newline> §<indent>§ pass <newline> <newline>
----------------------------------------
Token(input):  <newline>
Predicted:  §OOV§ (
Actual   :  §<indent>§ pass <newline> <newline> var821
----------------------------------------
Token(input):  §<indent>§
Predicted:  { §OOV§
Actual   :  pass <newline> <newline> var821 =
----------------------------------------
Token(input):  pass
Predicted:  §NUM§ <newline

----------------------------------------
Token(input):  .
Predicted:  §OOV§ (
Actual   :  = arg123 <newline> <newline>
----------------------------------------
Token(input):  (*) attribute|attribute1729
Predicted:  §OOV§ (
Actual   :  = arg123 <newline> <newline> §<dedent>§
----------------------------------------
Token(input):  =
Predicted:  §OOV§ (
Actual   :  arg123 <newline> <newline> §<dedent>§ def
----------------------------------------
Token(input):  arg123
Predicted:  create (
Actual   :  <newline> <newline> §<dedent>§ def function234
----------------------------------------
Token(input):  <newline>
Predicted:  §OOV§ (
Actual   :  <newline> §<dedent>§ def function234 (
----------------------------------------
Token(input):  <newline>
Predicted:  §OOV§ (
Actual   :  §<dedent>§ def function234 ( self
----------------------------------------
Token(input):  §<dedent>§
Predicted:  §OOV§ (
Actual   :  def function234 ( self ,
----------------------------------------
Token(input):  d